In [2]:
from langchain_community.document_loaders import CSVLoader
from langchain.docstore.document import Document 
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter

model="llama3"
llm = ChatOllama(model=model, temperature=0)

In [3]:
# loader = CSVLoader("./t20i_Matches_Data.csv")
# docs = loader.load()
import csv

columns_to_embed=["Match Name","Series Name", "Match Date","Team1 Name", "Team1 Runs Scored","Team1 Wickets Fell","Team2 Name","Team2 Runs Scored","Team2 Wickets Fell","Match Venue (Stadium)","Match Venue (City)","Match Venue (Country)","Umpire 1","Umpire 2","Match Referee","Toss Winner","Toss Winner Choice","Match Winner","Match Result Text"]
columns_to_metadata = ["Match Name","Series Name", "Match Date","Team1 Name", "Team1 Runs Scored","Team1 Wickets Fell","Team2 Name","Team2 Runs Scored","Team2 Wickets Fell","Match Venue (Stadium)","Match Venue (City)","Match Venue (Country)","Umpire 1","Umpire 2","Match Referee","Toss Winner","Toss Winner Choice","Match Winner","Match Result Text"]

docs = []
with open('t20i_Matches_Data.csv', newline="", encoding='utf-8-sig') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for i, row in enumerate(csv_reader):
        to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
        values_to_embed = {k: row[k] for k in columns_to_embed if k in row}
        to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
        newDoc = Document(page_content=to_embed, metadata=to_metadata)
        docs.append(newDoc)



In [4]:
splitter = CharacterTextSplitter(separator = "\n",
                                chunk_size=300, 
                                chunk_overlap=0,
                                length_function=len)
documents = splitter.split_documents(docs)

print(documents[0],'\n',
documents[1],'\n',
documents[2],'\n',
documents[3],'\n',
documents[4],'\n',
documents[5])

page_content='Match Name: Australia Vs India Only T20I\nSeries Name: India tour of Australia  - 2007 (2007/08)\nMatch Date: 2008-02-01\nTeam1 Name: India\nTeam1 Runs Scored: 74.0\nTeam1 Wickets Fell: 10.0\nTeam2 Name: Australia\nTeam2 Runs Scored: 75.0\nTeam2 Wickets Fell: 1.0' metadata={'Match Name': 'Australia Vs India Only T20I', 'Series Name': 'India tour of Australia  - 2007 (2007/08)', 'Match Date': '2008-02-01', 'Team1 Name': 'India', 'Team1 Runs Scored': '74.0', 'Team1 Wickets Fell': '10.0', 'Team2 Name': 'Australia', 'Team2 Runs Scored': '75.0', 'Team2 Wickets Fell': '1.0', 'Match Venue (Stadium)': 'Melbourne Cricket Ground', 'Match Venue (City)': 'Melbourne', 'Match Venue (Country)': 'Australia', 'Umpire 1': 'BNJ Oxenford', 'Umpire 2': 'SJA Taufel', 'Match Referee': 'JJ Crowe', 'Toss Winner': 'India', 'Toss Winner Choice': 'bat', 'Match Winner': 'Australia', 'Match Result Text': 'Australia won by 9 wickets (with 52 balls remaining)'} 
 page_content='Match Venue (Stadium): Mel

In [ ]:
# splitter1 = RecursiveCharacterTextSplitter(
#                                 chunk_size=300, 
#                                 chunk_overlap=0,
#                                 length_function=len)
# documents1 = splitter1.split_documents(docs)

# print(documents1[0],'\n',
# documents1[1],'\n',
# documents1[2],'\n',
# documents1[3],'\n',
# documents1[4],'\n',
# documents1[5])

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings,
)

print(len(vectorstore))

/Users/varun/learn/ai/learn-ai/env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/varun/learn/ai/learn-ai/env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/varun/learn/ai/learn-ai/env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when 

2539


In [6]:
vectorstore.similarity_search("Australia Vs Sri Lanka 3Rd T20I")

[Document(page_content='Match Name: Sri Lanka Vs Australia 2Nd T20I\nSeries Name: Australia tour of Sri Lanka  - 2016 (2016)\nMatch Date: 2016-09-09\nTeam1 Name: Sri Lanka\nTeam1 Runs Scored: 128.0\nTeam1 Wickets Fell: 9.0\nTeam2 Name: Australia\nTeam2 Runs Scored: 130.0\nTeam2 Wickets Fell: 6.0', metadata={'Match Date': '2016-09-09', 'Match Name': 'Sri Lanka Vs Australia 2Nd T20I', 'Match Referee': 'J Srinath', 'Match Result Text': 'Australia won by 4 wickets (with 13 balls remaining)', 'Match Venue (City)': 'Colombo', 'Match Venue (Country)': 'Sri Lanka', 'Match Venue (Stadium)': 'R Premadasa Stadium', 'Match Winner': 'Australia', 'Series Name': 'Australia tour of Sri Lanka  - 2016 (2016)', 'Team1 Name': 'Sri Lanka', 'Team1 Runs Scored': '128.0', 'Team1 Wickets Fell': '9.0', 'Team2 Name': 'Australia', 'Team2 Runs Scored': '130.0', 'Team2 Wickets Fell': '6.0', 'Toss Winner': 'Sri Lanka', 'Toss Winner Choice': 'bat', 'Umpire 1': 'REJ Martinesz', 'Umpire 2': 'RSA Palliyaguruge'}),
 Docu

In [50]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 6631},
# )

In [32]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="Match Name",
        description="Name of the match",
        type="string",
    ),
    AttributeInfo(
        name="Series Name",
        description="Name of the series",
        type="string",
    ),
    AttributeInfo(
        name="Match Date",
        description="Date on which the match took place. Ex. 2008-02-01 ",
        type="date",
    ),
    AttributeInfo(
        name="Team1 Name", 
        description="Among two teams played this is the name of the first team that is playing the match", type="string"
    ),
    AttributeInfo(
        name="Team1 Runs Scored", 
        description="Among two teams played this is the runs scored by the first team that played the match", 
        type="string"
    ),
    AttributeInfo(
        name="Team1 Wickets Fell", 
        description="Among two teams played this is the wickets fallen for the first team that played the match", 
        type="string"
    ),
    AttributeInfo(
        name="Team2 Name", 
        description="Among two teams played this is the name of the second team that is playing the match", type="string"
    ),
    AttributeInfo(
        name="Team2 Runs Scored", 
        description="Among two teams played this is the runs scored by the second team that played the match", 
        type="string"
    ),
    AttributeInfo(
        name="Team2 Wickets Fell", 
        description="Among two teams played this is the wickets fallen for the second team that played the match", 
        type="string"
    ),
     AttributeInfo(
        name="Match Venue (Stadium)", 
        description="Name of the stadium at which the match took place", 
        type="string"
    ),
    AttributeInfo(
        name="Match Venue (City)", 
        description="Name of the city at which the match took place", 
        type="string"
    ),
    AttributeInfo(
        name="Match Venue (Country)", 
        description="Name of the country at which the match took place", 
        type="string"
    ),
    AttributeInfo(
        name="Umpire 1", 
        description="Name of the first umpire", 
        type="string"
    ),
    AttributeInfo(
        name="Umpire 2", 
        description="Name of the second umpire", 
        type="string"
    ),
    AttributeInfo(
        name="Match Referee", 
        description="Name of the match referee", 
        type="string"
    ),
    AttributeInfo(
        name="Toss Winner", 
        description="Among two teams played this is the name of the team that won the toss", 
        type="string"
    ),
    AttributeInfo(
        name="Toss Winner Choice", 
        description="This is the choice made the team that won the toss. Ex. bat, bowl", 
        type="string"
    ),
    AttributeInfo(
        name="Match Winner", 
        description="Name of the team that won the match", 
        type="string"
    ),
    AttributeInfo(
        name="Match Result Text", 
        description="Description of the match result", 
        type="string"
    ),
]

document_content_description = "t20 cricket matches details"

retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True)


In [26]:
retriever.get_relevant_documents("Netherlands Vs Zimbabwe 2Nd T20I")

[Document(page_content='Match Name: Netherlands Vs Zimbabwe 2Nd T20I\nSeries Name: ZIM in IRELAND and NL  - 2019 (2019)\nMatch Date: 2019-06-25\nTeam1 Name: Netherlands\nTeam1 Runs Scored: 152.0\nTeam1 Wickets Fell: 8.0\nTeam2 Name: Zimbabwe\nTeam2 Runs Scored: 152.0\nTeam2 Wickets Fell: 10.0\nMatch Venue (Stadium): Hazelaarweg', metadata={'Match Date': '2019-06-25', 'Match Name': 'Netherlands Vs Zimbabwe 2Nd T20I', 'Match Referee': 'GF Labrooy', 'Match Result Text': 'Match tied (Zimbabwe won the one-over eliminator)', 'Match Venue (City)': 'Rotterdam', 'Match Venue (Country)': 'Netherlands', 'Match Venue (Stadium)': 'Hazelaarweg', 'Match Winner': '', 'Series Name': 'ZIM in IRELAND and NL  - 2019 (2019)', 'Team1 Name': 'Netherlands', 'Team1 Runs Scored': '152.0', 'Team1 Wickets Fell': '8.0', 'Team2 Name': 'Zimbabwe', 'Team2 Runs Scored': '152.0', 'Team2 Wickets Fell': '10.0', 'Toss Winner': 'Netherlands', 'Toss Winner Choice': 'bat', 'Umpire 1': 'HKG Jansen', 'Umpire 2': 'R Akram'}),
 

In [39]:
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm



In [43]:
response = rag_chain.invoke("which match took place at AMI Stadium")

print(response.content)

The match that took place at AMI Stadium is:

* New Zealand Vs Australia 2Nd T20I (2010-02-28)
* New Zealand Vs Pakistan 3Rd T20I (2010-12-30)
* New Zealand Vs India 1St T20I (2009-02-25)


In [34]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage
from operator import itemgetter
from langchain.chains.retrieval import  create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "The context provided has data regarding cricket matches played for years"
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)
# response = rag_chain.invoke("among all the matches that have happened, when did the first australia vs india match happen")
config = {"configurable": {"session_id": "first"}}



In [44]:
conversational_rag_chain.invoke(
    {"input": "which match took place at AMI Stadium"},
    config={
        "configurable": {"session_id": "session-1"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run 449ae316-3408-4193-bbb3-6378d00f4e71 not found for run 89fb1a0b-93c3-4827-9c34-fce569b04e68. Treating as a root run.


'According to the context, there were three matches that took place at AMI Stadium:\n\n1. The toss winner chose to bat.\n2. Pakistan won by 103 runs.\n3. New Zealand won by 7 wickets (with 7 balls remaining).\n\nSo, all three matches took place at AMI Stadium in Christchurch, New Zealand.'

In [21]:
conversational_rag_chain.invoke(
    {"input": "what are the total number of matches played"},
    config={
        "configurable": {"session_id": "session-1"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run 0f61c5aa-4cac-4755-9185-b854a205c048 not found for run bf313cbd-f83c-4d1f-95f9-be3458d23f9a. Treating as a root run.


"I don't know. The provided context only contains information about matches between Philippines and P.N.G., but it doesn't mention any matches involving Australia or Sri Lanka."

In [ ]:
vectorstore.delete_collection()